In [1]:
import time, os, math, numpy as np, pandas as pd, sklearn as sk, matplotlib as plt

In [2]:
df_arcos = pd.read_csv('data/arcos/arcos_all_washpost.tsv', delimiter='\t', nrows=1000000)

# Load dataset for all Primary Care HPSA destinations with one record per facility
df_hpsa_pc = pd.read_csv("data/hpsa/BCD_HPSA_FCT_DET_PC.csv")

# Load dataset for all Mental Health HPSA destinations with one record per facility
df_hpsa_mh = pd.read_csv('data/hpsa/BCD_HPSA_FCT_DET_MH.csv')

# Load metadata for all columns - column descriptions
df_metadata = pd.read_excel('data/hpsa/HPSA_DATAMART_METADATA.XLSX')

# Load dataset for medically underserved areas
df_mua = pd.read_csv('data/hpsa/MUA_DET.csv')

# Load dataset for autoscoring - mental health
df_autoscoring_mh = pd.read_csv('data/hpsa/SDMS_AUTO_HPSA_SITE_MH.csv')

# Load dataset for autoscoring - primary care
df_autoscoring_pc = pd.read_csv('data/hpsa/SDMS_AUTO_HPSA_SITE_PC.csv')

<ipython-input-2-4d4889cfed16>:1: DtypeWarning: Columns (3,5,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_arcos = pd.read_csv('data/arcos/arcos_all_washpost.tsv', delimiter='\t', nrows=1000000)


In [3]:
df_arcos.head()

,REPORTER_DEA_NO,REPORTER_BUS_ACT,REPORTER_NAME,REPORTER_ADDL_CO_INFO,REPORTER_ADDRESS1,REPORTER_ADDRESS2,REPORTER_CITY,REPORTER_STATE,REPORTER_ZIP,REPORTER_COUNTY,...,DRUG_NAME,Measure,MME_Conversion_Factor,Dosage_Strength,TRANSACTION_DATE,Combined_Labeler_Name,Reporter_family,CALC_BASE_WT_IN_GM,DOSAGE_UNIT,MME
0,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-02-08,"Par Pharmaceutical, Inc.",McKesson Corporation,0.89650,100.0,1344.75
1,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,7.5,2011-03-07,SpecGx LLC,McKesson Corporation,0.45405,100.0,454.05
2,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,OXYCODONE,TAB,1.5,10.0,2011-03-10,"Par Pharmaceutical, Inc.",McKesson Corporation,3.58600,400.0,5379.00
3,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-05,SpecGx LLC,McKesson Corporation,1.51350,500.0,1513.50
4,RM0220688,DISTRIBUTOR,MCKESSON CORPORATION,NaN,DBA MCKESSON DRUG CO.,3000 KENSKILL AVE,WASHINGTON CT HOUSE,OH,43160,FAYETTE,...,HYDROCODONE,TAB,1.0,5.0,2011-04-06,SpecGx LLC,McKesson Corporation,0.30270,100.0,302.70


In [4]:
df_arcos.columns

Index(['REPORTER_DEA_NO', 'REPORTER_BUS_ACT', 'REPORTER_NAME',
       'REPORTER_ADDL_CO_INFO', 'REPORTER_ADDRESS1', 'REPORTER_ADDRESS2',
       'REPORTER_CITY', 'REPORTER_STATE', 'REPORTER_ZIP', 'REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'Measure', 'MME_Conversion_Factor', 'Dosage_Strength',
       'TRANSACTION_DATE', 'Combined_Labeler_Name', 'Reporter_family',
       'CALC_BASE_WT_IN_GM', 'DOSAGE_UNIT', 'MME'],
      dtype='object')

In [5]:
df_arcos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   REPORTER_DEA_NO        1000000 non-null  object 
 1   REPORTER_BUS_ACT       1000000 non-null  object 
 2   REPORTER_NAME          1000000 non-null  object 
 3   REPORTER_ADDL_CO_INFO  216186 non-null   object 
 4   REPORTER_ADDRESS1      1000000 non-null  object 
 5   REPORTER_ADDRESS2      274977 non-null   object 
 6   REPORTER_CITY          1000000 non-null  object 
 7   REPORTER_STATE         1000000 non-null  object 
 8   REPORTER_ZIP           1000000 non-null  int64  
 9   REPORTER_COUNTY        1000000 non-null  object 
 10  BUYER_DEA_NO           1000000 non-null  object 
 11  BUYER_BUS_ACT          1000000 non-null  object 
 12  BUYER_NAME             1000000 non-null  object 
 13  BUYER_ADDL_CO_INFO     441084 non-null   object 
 14  BUYER_ADDRESS1     

In [8]:
df_arcos.nunique().reset_index().sort_values(0, ascending=False)

,index,0
27,TRANSACTION_DATE,3898
10,BUYER_DEA_NO,3686
14,BUYER_ADDRESS1,3535
12,BUYER_NAME,2393
22,NDC_NO,1596
13,BUYER_ADDL_CO_INFO,1552
15,BUYER_ADDRESS2,708
18,BUYER_ZIP,648
32,MME,572
30,CALC_BASE_WT_IN_GM,557


In [11]:
df_arcos['BUYER_NAME'].value_counts().reset_index().head(25)

,BUYER_NAME,count
0,WALGREEN CO.,138366
1,"HOOK-SUPERX, L.L.C.",98235
2,"KENTUCKY CVS PHARMACY, L.L.C.",32293
3,"RITE AID OF KENTUCKY, INC.",26850
4,KROGER PHARMACY,19715
5,WALGREEN CO,13888
6,TARGET STORES A DIV.OF TARGET CORP.,6176
7,MARSH DRUGS,4166
8,"RITE AID OF INDIANA, INC.",3279
9,WIGGINS DRUGS LIMITED,2995


In [13]:
df_arcos['DRUG_NAME'].value_counts().reset_index()

,DRUG_NAME,count
0,HYDROCODONE,537311
1,OXYCODONE,462689


In [3]:
df_hpsa_pc.head()

,HPSA Name,HPSA ID,Designation Type,HPSA Discipline Class,HPSA Score,PC MCTA Score,Primary State Abbreviation,HPSA Status,HPSA Designation Date,HPSA Designation Last Update Date,...,Rural Status Code,State Abbreviation,State and County Federal Information Processing Standard Code,State FIPS Code,State Name,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,Data Warehouse Record Create Date,Data Warehouse Record Create Date Text,Unnamed: 65
0,East Toledo,1396315268,High Needs Geographic HPSA,Primary Care,15,18.0,OH,Designated,01/31/1992,09/09/2021,...,N,OH,39095,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
1,East Toledo,1396315268,High Needs Geographic HPSA,Primary Care,15,18.0,OH,Designated,01/31/1992,09/09/2021,...,N,OH,39095,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
2,East Toledo,1396315268,High Needs Geographic HPSA,Primary Care,15,18.0,OH,Designated,01/31/1992,09/09/2021,...,N,OH,39095,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
3,Noble Correctional Institution,1396293612,Correctional Facility,Primary Care,6,0.0,OH,Designated,08/25/1999,03/15/2022,...,R,OH,39121,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
4,EDGERTON FAMILY HEALTH CENTER,1396199989,Rural Health Clinic,Primary Care,14,15.0,OH,Designated,12/16/2022,12/16/2022,...,R,OH,39171,39,Ohio,N,N,04/09/2024,2024/04/09,NaN


In [4]:
df_hpsa_pc.columns

Index(['HPSA Name', 'HPSA ID', 'Designation Type', 'HPSA Discipline Class',
       'HPSA Score', 'PC MCTA Score', 'Primary State Abbreviation',
       'HPSA Status', 'HPSA Designation Date',
       'HPSA Designation Last Update Date', 'Metropolitan Indicator',
       'HPSA Geography Identification Number', 'HPSA Degree of Shortage',
       'Withdrawn Date', 'HPSA FTE', 'HPSA Designation Population',
       '% of Population Below 100% Poverty', 'HPSA Formal Ratio',
       'HPSA Population Type', 'Rural Status', 'Longitude', 'Latitude',
       'BHCMIS Organization Identification Number', 'Break in Designation',
       'Common County Name', 'Common Postal Code', 'Common Region Name',
       'Common State Abbreviation', 'Common State County FIPS Code',
       'Common State FIPS Code', 'Common State Name', 'County Equivalent Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'Discipline Class Number', 'HPSA Address', 'HPSA City',
       'HPSA Co

In [5]:
df_hpsa_pc[['HPSA Degree of Shortage',
       'Withdrawn Date', 'HPSA FTE', 'HPSA Designation Population',
       '% of Population Below 100% Poverty', 'HPSA Formal Ratio',
       'HPSA Population Type', 'Rural Status', 'Longitude', 'Latitude',
       'BHCMIS Organization Identification Number', 'Break in Designation',
       'Common County Name', 'Common Postal Code', 'Common Region Name',
       'Common State Abbreviation', 'Common State County FIPS Code',
       'Common State FIPS Code', 'Common State Name', 'County Equivalent Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'Discipline Class Number', 'HPSA Address', 'HPSA City']]

,HPSA Degree of Shortage,Withdrawn Date,HPSA FTE,HPSA Designation Population,% of Population Below 100% Poverty,HPSA Formal Ratio,HPSA Population Type,Rural Status,Longitude,Latitude,...,Common Region Name,Common State Abbreviation,Common State County FIPS Code,Common State FIPS Code,Common State Name,County Equivalent Name,County or County Equivalent Federal Information Processing Standard Code,Discipline Class Number,HPSA Address,HPSA City
0,Not applicable,NaN,0.4000,24780.0,37.3,61950:1,Geographic Population,Non-Rural,NaN,NaN,...,Region 5,OH,39095,39,Ohio,Lucas,095,1,NaN,NaN
1,Not applicable,NaN,0.4000,24780.0,37.3,61950:1,Geographic Population,Non-Rural,NaN,NaN,...,Region 5,OH,39095,39,Ohio,Lucas,095,1,NaN,NaN
2,Not applicable,NaN,0.4000,24780.0,37.3,61950:1,Geographic Population,Non-Rural,NaN,NaN,...,Region 5,OH,39095,39,Ohio,Lucas,095,1,NaN,NaN
3,6,NaN,0.4500,2663.0,NaN,NaN,NaN,Rural,-81.550901,39.740703,...,Region 5,OH,39121,39,Ohio,Noble,121,1,15708 Mcconnellsville Rd,Caldwell
4,NaN,NaN,NaN,43256.0,NaN,NaN,NaN,Rural,-84.752887,41.448663,...,Region 5,OH,39171,39,Ohio,Williams,171,1,104 N West St,Edgerton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66853,Not applicable,NaN,0.0000,3320.0,12.8,NaN,Geographic Population,Rural,NaN,NaN,...,Region 6,TX,48295,48,Texas,Lipscomb,295,1,NaN,NaN
66854,NaN,02/05/2024,NaN,32983.0,NaN,NaN,NaN,Rural,-102.653797,32.721884,...,Region 6,TX,48165,48,Texas,Gaines,165,1,209 NW 8th St,Seminole
66855,Not applicable,NaN,0.0000,2983.0,15.6,NaN,Geographic Population,Rural,NaN,NaN,...,Region 6,TX,48413,48,Texas,Schleicher,413,1,NaN,NaN
66856,Not applicable,NaN,0.3160,1245.0,11.7,3940:1,Low Income Population HPSA,Rural,NaN,NaN,...,Region 6,TX,48023,48,Texas,Baylor,023,1,NaN,NaN


In [6]:
df_hpsa_mh.head()

,HPSA Name,HPSA ID,Designation Type,HPSA Discipline Class,HPSA Score,Primary State Abbreviation,HPSA Status,HPSA Designation Date,HPSA Designation Last Update Date,Metropolitan Indicator,...,Rural Status Code,State Abbreviation,State and County Federal Information Processing Standard Code,State FIPS Code,State Name,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,Data Warehouse Record Create Date,Data Warehouse Record Create Date Text,Unnamed: 64
0,MHCA 32,7396813432,Geographic HPSA,Mental Health,16,OH,Designated,03/14/1980,09/02/2021,Unknown,...,R,OH,39115,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
1,MHCA 32,7396813432,Geographic HPSA,Mental Health,16,OH,Designated,03/14/1980,09/02/2021,Unknown,...,R,OH,39119,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
2,Noble Correctional Institution,7396718116,Correctional Facility,Mental Health,21,OH,Designated,09/28/2010,03/15/2022,Unknown,...,R,OH,39121,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
3,Putnam County,7396677835,Geographic HPSA,Mental Health,14,OH,Designated,08/09/2010,09/02/2021,Unknown,...,R,OH,39137,39,Ohio,N,N,04/09/2024,2024/04/09,NaN
4,LI-Cuyahoga County,7396637206,HPSA Population,Mental Health,18,OH,Designated,03/22/2022,03/22/2022,Unknown,...,N,OH,39035,39,Ohio,N,N,04/09/2024,2024/04/09,NaN


In [7]:
df_mua.head()

,MUA/P ID,MUA/P Area Code,MUA/P Service Area Name,Designation Type Code,Designation Type,MUA/P Status Code,MUA/P Status Description,Designation Date,MUA/P Designation Date String,MUA/P Update Date,...,Common Region Code,Common Region Name,Common State Name,Common State Abbreviation,Common State FIPS Code,Common State County FIPS Code,Common County Name,Data Warehouse Record Create Date,Data Warehouse Record Create Date Text,Unnamed: 64
0,6112,13245000700,Eastern Augusta Service Area,MUA,Medically Underserved Area,D,Designated,06/30/1999,1999/06/30,06/30/1999,...,4,Region 4,Georgia,GA,13,13245,"Richmond County, GA",04/09/2024,2024/04/09,NaN
1,6112,13245000900,Eastern Augusta Service Area,MUA,Medically Underserved Area,D,Designated,06/30/1999,1999/06/30,06/30/1999,...,4,Region 4,Georgia,GA,13,13245,"Richmond County, GA",04/09/2024,2024/04/09,NaN
2,6112,13245001400,Eastern Augusta Service Area,MUA,Medically Underserved Area,D,Designated,06/30/1999,1999/06/30,06/30/1999,...,4,Region 4,Georgia,GA,13,13245,"Richmond County, GA",04/09/2024,2024/04/09,NaN
3,6112,13245001500,Eastern Augusta Service Area,MUA,Medically Underserved Area,D,Designated,06/30/1999,1999/06/30,06/30/1999,...,4,Region 4,Georgia,GA,13,13245,"Richmond County, GA",04/09/2024,2024/04/09,NaN
4,6112,13245011000,Eastern Augusta Service Area,MUA,Medically Underserved Area,D,Designated,06/30/1999,1999/06/30,06/30/1999,...,4,Region 4,Georgia,GA,13,13245,"Richmond County, GA",04/09/2024,2024/04/09,NaN


In [8]:
df_autoscoring_mh.head()

,HPSA Source Identification Number,BHCMIS Organization Identification Number,Auto-HPSA Facility Site Source Identification Number,Site Source Identification Number Type,HPSA Name,Auto-HPSA Facility Site Name,Auto-HPSA Facility Site Address,Auto-HPSA Facility Site City,Auto-HPSA Facility Site State,Auto-HPSA Facility Site Postal Code,Auto-HPSA Facility Source Create Date Text,Auto-HPSA Facility Source Last Review Date Text,Data Warehouse Record Create Date Text,Unnamed: 13
0,7011347136,NaN,013960,CMS Certification Number (CCN),WETUMPKA FAMILY RURAL HEALTH CLINIC,WETUMPKA FAMILY RURAL HEALTH CLINIC,815 Jackson Trace Rd,Wetumpka,AL,36092-1504,2021/09/11,2021/09/11,2023/07/06,NaN
1,7011587068,NaN,013846,CMS Certification Number (CCN),CULLMAN INTERNAL MEDICINE PC,CULLMAN INTERNAL MEDICINE PC,1890 Al Highway 157 Ste 300,Cullman,AL,35058-0689,2019/08/18,2019/08/18,2023/07/06,NaN
2,7012366708,04E01383,BPS-LAL-030265,BPHC Assigned Number,Happi Health,HAPPI Behavioral Health,2327 Pansy St SW Ste B,Huntsville,AL,35801-3804,2021/09/11,2021/09/11,2023/07/06,NaN
3,7012366708,04E01383,BPS-LAL-030264,BPHC Assigned Number,Happi Health,Sparkman Family Practice &amp; Urgent Care,2597 Sparkman Dr NW,Huntsville,AL,35810-3868,2021/09/11,2021/09/11,2023/07/06,NaN
4,7012366708,04E01383,BPS-LAL-030270,BPHC Assigned Number,Happi Health,HAPPI Pediatric &amp; Family Care,813 Franklin St SE,Huntsville,AL,35801-4311,2021/09/11,2021/09/11,2023/07/06,NaN


In [10]:
df_autoscoring_pc.head()

,HPSA Source Identification Number,BHCMIS Organization Identification Number,Auto-HPSA Facility Site Source Identification Number,Site Source Identification Number Type,HPSA Name,Auto-HPSA Facility Site Name,Auto-HPSA Facility Site Address,Auto-HPSA Facility Site City,Auto-HPSA Facility Site State,Auto-HPSA Facility Site Postal Code,Auto-HPSA Facility Source Create Date Text,Auto-HPSA Facility Source Last Review Date Text,Data Warehouse Record Create Date Text,Unnamed: 13
0,1012149391,04E01383,BPS-LAL-030270,BPHC Assigned Number,Happi Health,HAPPI Pediatric &amp; Family Care,813 Franklin St SE,Huntsville,AL,35801-4311,2021/09/11,2021/09/11,2023/07/06,NaN
1,1012149391,04E01383,BPS-LAL-030264,BPHC Assigned Number,Happi Health,Sparkman Family Practice &amp; Urgent Care,2597 Sparkman Dr NW,Huntsville,AL,35810-3868,2021/09/11,2021/09/11,2023/07/06,NaN
2,1012149391,04E01383,BPS-LAL-030265,BPHC Assigned Number,Happi Health,HAPPI Behavioral Health,"2327 Pansy St SW, #B",Huntsville,AL,35801-3804,2021/09/11,2021/09/11,2023/07/06,NaN
3,1012149391,04E01383,BPS-LAL-030268,BPHC Assigned Number,Happi Health,HAPPI Urgent Care,807 Franklin St SE,Huntsville,AL,35801-4311,2021/09/11,2021/09/11,2023/07/06,NaN
4,1012320208,NaN,013457,CMS Certification Number (CCN),BUTLER PRIMARY CARE,BUTLER PRIMARY CARE,300 N College St,Greenville,AL,36037-2025,2022/08/04,2022/08/04,2023/07/06,NaN


In [11]:
df_metadata

,Attribute Name,Physical Column Name,User Friendly Names,Definition,Data Restriction,Organization,Update Frequency,Data Type,Business Rule / Format,HDW Transformation,Valid Values,Definition Update Date
0,Congressional District Name,CONG_DIST_NM,Congressional District Name; Cong. District; C...,This attribute represents the name of the U.S....,Public,U.S. House of Representatives,As Required,varchar(50),None.,(1) For states with two or more representative...,Expected values are text strings for Congressi...,2016/05/27
1,State FIPS Code,STATE_FIPS_CD,State FIPS Code; State Federal Information Pro...,This attribute represents the State Federal In...,Public,National Institute of Standards and Technology...,As Required,varchar(2),None.,This attribute is a common element which may b...,Expected values are two character numeric text...,2020/08/15
2,State Name,STATE_NM,State Name; Component State; Component State N...,This attribute represents the state or territo...,Public,National Institute of Standards and Technology...,As Required,varchar(50),None.,This attribute is a common element which may b...,Expected values are alphabetic text strings re...,2020/08/15
3,U.S. Senator Name (2),US_SENATE_NM2,U.S. Senator Name (2); Name of U.S. Senator; N...,This attribute represents the name of one of t...,Public,U.S. Senate,Monthly,varchar(100),The District of Columbia and the U.S. territor...,This attribute is a common element which may b...,Expected values are text strings containing th...,2018/03/16
4,Data Warehouse Record Create Date,DW_RECORD_CREATE_DT,Data Warehouse Record Create Date; Data As-of-...,This attribute represents the date on which th...,Public,HRSA Data Warehouse,As Required,smalldatetime,None.,(1) System date on which the data set was refr...,Expected values are date time where the time i...,2022/07/19
...,...,...,...,...,...,...,...,...,...,...,...,...
95,American Recovery and Reinvestment Act of 2009...,ARRA_DESC,American Recovery and Reinvestment Act of 2009...,This attribute represents the indication of wh...,Public,HRSA Data Warehouse,Daily,varchar(5),None.,None.,Expected values are empty character strings. C...,2016/01/04
96,Indian Health Service Site Status,IHS_DESC,Indian Health Service Site Status; Is IHS Desi...,This attribute represents the Indian Health Se...,Public,HRSA Data Warehouse,On Demand,varchar(5),None.,This value is the decoded Indian Health Servic...,Expected values are single character text stri...,2016/01/04
97,Federal Bureau of Prisons Site Status,BOP_DESC,Federal Bureau of Prisons Site Status; Is BOP ...,This attribute represents the Federal Bureau o...,Public,HRSA Data Warehouse,Daily,varchar(5),None.,This value is the decoded Federal Bureau of Pr...,Expected values are single character text stri...,2016/01/04
98,U.S. Immigration and Customs Enforcement Site ...,ICE_DESC,U.S. Immigration and Customs Enforcement Site ...,This attribute represents the U.S. Immigration...,Public,HRSA Data Warehouse,On Demand,varchar(5),None.,This value is the decoded U.S. Immigration and...,Expected values are single character text stri...,2016/01/04


In [77]:
# Load 2019 FBI NIBRS data

df_nibrs_19 = pd.read_excel('data/United_States_Offense_Type_by_Agency_2019.xls', skiprows=[0,1,2])

df_nibrs_19.head(25)

,State,Agency Type,Agency Name,Population1,Total\nOffenses,Crimes\nAgainst\nPersons,Crimes\nAgainst\nProperty,Crimes\nAgainst\nSociety,Crimes Against Persons,Unnamed: 9,...,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assault\nOffenses,Aggravated\nAssault,...,Betting/\nWagering,Operating/\nPromoting/\nAssisting\nGambling,Gambling\nEquipment\nViolations,Sports\nTampering,Por-\nnography/\nObscene\nMaterial,Pros-\ntitution\nOffenses,Pros-\ntitution,Assisting or\nPromoting\nProstitution,Purchasing\nProstitution,Weapon\nLaw\nViolations
1,ALABAMA,Cities,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811,68,...,0,0,0,0,2,1,1,0,0,67
2,ARIZONA,Cities,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383,71,...,0,0,0,0,6,0,0,0,0,36
3,NaN,NaN,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214,61,...,0,0,0,0,1,0,0,0,0,18
4,NaN,NaN,Eagar,4897.0,102.0,36.0,29.0,37.0,21,7,...,0,0,0,0,0,0,0,0,0,1
5,NaN,NaN,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642,157,...,0,0,0,0,47,10,10,0,0,78
6,NaN,NaN,Huachuca City,1723.0,73.0,26.0,41.0,6.0,20,1,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,Somerton,16771.0,257.0,70.0,155.0,32.0,57,15,...,0,0,0,0,1,0,0,0,0,5
8,NaN,NaN,Wellton,3048.0,105.0,26.0,58.0,21.0,24,4,...,0,0,0,0,0,0,0,0,0,1
9,NaN,NaN,Yuma,98769.0,7037.0,1110.0,4575.0,1352.0,965,309,...,0,0,0,0,30,2,0,2,0,93


In [78]:
# Get missing column names
names = df_nibrs_19.iloc[0].tolist()
missing_names = names[8:]
# missing_names = [name.replace('\n', ' ').replace(' ', '_').replace('/_', '/').replace('-_', '').replace('__', '_') for name in missing_names]

#Get column names and replace UnNamed columns with the corresponding name
column_names = df_nibrs_19.columns.to_list()
column_names = column_names[:8]
for name in missing_names:
    column_names.append(name)
    
column_names = [name.replace('\n', ' ').replace(' ', '_').replace('/_', '/').replace('-_', '').replace('__', '_') for name in column_names]
df_nibrs_19.columns = column_names
df_nibrs_19 = df_nibrs_19.drop(df_nibrs_19.index[0])
df_nibrs_19.head()

,State,Agency_Type,Agency_Name,Population1,Total_Offenses,Crimes_Against_Persons,Crimes_Against_Property,Crimes_Against_Society,Assault_Offenses,Aggravated_Assault,...,Betting/Wagering,Operating/Promoting/Assisting_Gambling,Gambling_Equipment_Violations,Sports_Tampering,Pornography/Obscene_Material,Prostitution_Offenses,Prostitution,Assisting_or_Promoting_Prostitution,Purchasing_Prostitution,Weapon_Law_Violations
1,ALABAMA,Cities,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811,68,...,0,0,0,0,2,1,1,0,0,67
2,ARIZONA,Cities,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383,71,...,0,0,0,0,6,0,0,0,0,36
3,NaN,NaN,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214,61,...,0,0,0,0,1,0,0,0,0,18
4,NaN,NaN,Eagar,4897.0,102.0,36.0,29.0,37.0,21,7,...,0,0,0,0,0,0,0,0,0,1
5,NaN,NaN,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642,157,...,0,0,0,0,47,10,10,0,0,78


In [79]:
# Replace NaN values in 'State' with the first non-NaN value using ffill
df_nibrs_19['State'] = df_nibrs_19['State'].fillna(method='ffill')

# Replace NaN values in 'Agency_Type' with the first non-NaN value using ffill
df_nibrs_19['Agency_Type'] = df_nibrs_19['Agency_Type'].fillna(method='ffill')

df_nibrs_19.head(20)

,State,Agency_Type,Agency_Name,Population1,Total_Offenses,Crimes_Against_Persons,Crimes_Against_Property,Crimes_Against_Society,Assault_Offenses,Aggravated_Assault,...,Betting/Wagering,Operating/Promoting/Assisting_Gambling,Gambling_Equipment_Violations,Sports_Tampering,Pornography/Obscene_Material,Prostitution_Offenses,Prostitution,Assisting_or_Promoting_Prostitution,Purchasing_Prostitution,Weapon_Law_Violations
1,ALABAMA,Cities,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811,68,...,0,0,0,0,2,1,1,0,0,67
2,ARIZONA,Cities,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383,71,...,0,0,0,0,6,0,0,0,0,36
3,ARIZONA,Cities,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214,61,...,0,0,0,0,1,0,0,0,0,18
4,ARIZONA,Cities,Eagar,4897.0,102.0,36.0,29.0,37.0,21,7,...,0,0,0,0,0,0,0,0,0,1
5,ARIZONA,Cities,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642,157,...,0,0,0,0,47,10,10,0,0,78
6,ARIZONA,Cities,Huachuca City,1723.0,73.0,26.0,41.0,6.0,20,1,...,0,0,0,0,0,0,0,0,0,0
7,ARIZONA,Cities,Somerton,16771.0,257.0,70.0,155.0,32.0,57,15,...,0,0,0,0,1,0,0,0,0,5
8,ARIZONA,Cities,Wellton,3048.0,105.0,26.0,58.0,21.0,24,4,...,0,0,0,0,0,0,0,0,0,1
9,ARIZONA,Cities,Yuma,98769.0,7037.0,1110.0,4575.0,1352.0,965,309,...,0,0,0,0,30,2,0,2,0,93
10,ARIZONA,Metropolitan Counties,Yuma,NaN,2987.0,742.0,1862.0,383.0,688,109,...,0,0,0,0,7,0,0,0,0,35


In [80]:
df_nibrs_19 = df_nibrs_19[df_nibrs_19["Agency_Type"]=="Cities"]
df_nibrs_19.head(20)

,State,Agency_Type,Agency_Name,Population1,Total_Offenses,Crimes_Against_Persons,Crimes_Against_Property,Crimes_Against_Society,Assault_Offenses,Aggravated_Assault,...,Betting/Wagering,Operating/Promoting/Assisting_Gambling,Gambling_Equipment_Violations,Sports_Tampering,Pornography/Obscene_Material,Prostitution_Offenses,Prostitution,Assisting_or_Promoting_Prostitution,Purchasing_Prostitution,Weapon_Law_Violations
1,ALABAMA,Cities,Hoover,85670.0,4293.0,840.0,2801.0,652.0,811,68,...,0,0,0,0,2,1,1,0,0,67
2,ARIZONA,Cities,Apache Junction2,42531.0,NaN,430.0,NaN,487.0,383,71,...,0,0,0,0,6,0,0,0,0,36
3,ARIZONA,Cities,Coolidge,13138.0,1155.0,230.0,684.0,241.0,214,61,...,0,0,0,0,1,0,0,0,0,18
4,ARIZONA,Cities,Eagar,4897.0,102.0,36.0,29.0,37.0,21,7,...,0,0,0,0,0,0,0,0,0,1
5,ARIZONA,Cities,Gilbert,253619.0,10589.0,1844.0,6144.0,2601.0,1642,157,...,0,0,0,0,47,10,10,0,0,78
6,ARIZONA,Cities,Huachuca City,1723.0,73.0,26.0,41.0,6.0,20,1,...,0,0,0,0,0,0,0,0,0,0
7,ARIZONA,Cities,Somerton,16771.0,257.0,70.0,155.0,32.0,57,15,...,0,0,0,0,1,0,0,0,0,5
8,ARIZONA,Cities,Wellton,3048.0,105.0,26.0,58.0,21.0,24,4,...,0,0,0,0,0,0,0,0,0,1
9,ARIZONA,Cities,Yuma,98769.0,7037.0,1110.0,4575.0,1352.0,965,309,...,0,0,0,0,30,2,0,2,0,93
11,ARKANSAS,Cities,Alexander,3315.0,216.0,55.0,113.0,48.0,48,9,...,0,0,0,0,0,0,0,0,0,1


In [82]:
print(df_nibrs_19.shape)
df_nibrs_19.info()

(4168, 69)
<class 'pandas.core.frame.DataFrame'>
Index: 4168 entries, 1 to 6582
Data columns (total 69 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   State                                        4168 non-null   object 
 1   Agency_Type                                  4168 non-null   object 
 2   Agency_Name                                  4168 non-null   object 
 3   Population1                                  4168 non-null   float64
 4   Total_Offenses                               4143 non-null   float64
 5   Crimes_Against_Persons                       4165 non-null   float64
 6   Crimes_Against_Property                      4146 non-null   float64
 7   Crimes_Against_Society                       4168 non-null   float64
 8   Assault_Offenses                             4168 non-null   object 
 9   Aggravated_Assault                           4168 non-null   object 